# バギング（Bagging: Bootstrap Aggregating）
複数のサブデータセット（元のデータからブートストラップサンプリングによって抽出されたデータ）を用いて個別にモデルを学習し、その予測結果を平均（回帰の場合）または多数決（分類の場合）で統合する  
ランダムフォレストは、バギングの代表例  
各モデルが独立して学習されるため、過学習のリスクが低減される  
ノイズの影響が平均化され、安定した予測が期待できる  
データのばらつきやノイズに対してロバスト

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# シード固定
torch.manual_seed(42)
np.random.seed(42)

# サンプルデータ生成： y = 2*x + 1 にノイズを加える
X = np.linspace(-1, 1, 100).reshape(-1, 1)
y = 2 * X + 1 + np.random.normal(0, 0.1, X.shape)

# NumPy → Tensor
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

# シンプルなMLPモデル（1層隠れ層）
class SimpleMLP(nn.Module):
    def __init__(self, hidden_size=16):
        super(SimpleMLP, self).__init__()
        self.fc1 = nn.Linear(1, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# シンプルな学習ループ（エポック数は100）
def train_model(model, optimizer, epochs=100):
    criterion = nn.MSELoss()
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        preds = model(X_tensor)
        loss = criterion(preds, y_tensor)
        loss.backward()
        optimizer.step()
    return loss.item()


In [6]:
def bootstrap_sample(X, y):
    # ブートストラップサンプリング
    n = X.shape[0]
    indices = np.random.choice(n, n, replace=True)
    return X[indices], y[indices]

num_models = 5
bagged_models = []
for i in range(num_models):
    # ブートストラップサンプルを作成
    X_bs, y_bs = bootstrap_sample(X_tensor, y_tensor)
    # 各モデルを初期化
    model = SimpleMLP(hidden_size=16)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    # サンプル用のデータを使って訓練（ここでは100エポック）
    criterion = nn.MSELoss()
    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        preds = model(X_bs)
        loss = criterion(preds, y_bs)
        loss.backward()
        optimizer.step()
    bagged_models.append(model)

# 予測：各モデルの予測を平均
def bagging_predict(x):
    preds = []
    for model in bagged_models:
        model.eval()           # 各モデルを評価モードに切り替える
        with torch.no_grad():
            pred = model(x)    # 予測値を計算
        preds.append(pred)
    preds = torch.stack(preds, dim=0)
    return preds.mean(dim=0)

bagging_preds = bagging_predict(X_tensor)
print("Bagging ensemble final loss:", nn.MSELoss()(bagging_preds, y_tensor).item())


Bagging ensemble final loss: 0.008324990049004555
